# Functional test with FTDI
- https://www.silabs.com/documents/public/data-sheets/Si5351-B.pdf
- https://www.silabs.com/documents/public/application-notes/AN619.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', 'ORM', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from clock_generators.si535x.si5351 import Si5351 
from utilities.adapters import peripherals

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [5]:
cls = Si5351

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [6]:
with_hardware_device = True

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c() 
    
else:
    _i2c =  None  # using None for testing without actual hardware device.

si = cls(_i2c)  

In [7]:
si.map.save_to_file('Initial_values.txt')

clock_builder_values = si.map.read_file('Si5351A-RevB-Registers.txt') 
initial_values = si.map.read_file('Initial_values.txt')

In [8]:
si.map.addressed_values

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (9, 0),
 (15, 0),
 (16, 79),
 (17, 79),
 (18, 79),
 (19, 79),
 (20, 79),
 (21, 79),
 (22, 79),
 (23, 79),
 (24, 0),
 (25, 0),
 (26, 0),
 (27, 1),
 (28, 0),
 (29, 16),
 (30, 0),
 (31, 0),
 (32, 0),
 (33, 0),
 (34, 0),
 (35, 1),
 (36, 0),
 (37, 16),
 (38, 0),
 (39, 0),
 (40, 0),
 (41, 0),
 (42, 0),
 (43, 1),
 (44, 0),
 (45, 16),
 (46, 0),
 (47, 0),
 (48, 0),
 (49, 0),
 (50, 0),
 (51, 1),
 (52, 0),
 (53, 16),
 (54, 0),
 (55, 0),
 (56, 0),
 (57, 0),
 (58, 0),
 (59, 1),
 (60, 0),
 (61, 16),
 (62, 0),
 (63, 0),
 (64, 0),
 (65, 0),
 (66, 0),
 (67, 1),
 (68, 0),
 (69, 16),
 (70, 0),
 (71, 0),
 (72, 0),
 (73, 0),
 (74, 0),
 (75, 1),
 (76, 0),
 (77, 16),
 (78, 0),
 (79, 0),
 (80, 0),
 (81, 0),
 (82, 0),
 (83, 1),
 (84, 0),
 (85, 16),
 (86, 0),
 (87, 0),
 (88, 0),
 (89, 0),
 (90, 36),
 (91, 36),
 (92, 0),
 (149, 0),
 (150, 0),
 (151, 0),
 (152, 0),
 (153, 0),
 (154, 0),
 (155, 0),
 (156, 0),
 (157, 0),
 (158, 0),
 (159, 0),
 (160, 0),
 (161, 0),
 (162, 0),
 (1

In [9]:
si.map.reset()
si.read_all_registers()
read_back_values = si.map.addressed_values 

# mini = Si5351_mini(_i2c)
# mini.write_all_registers(av1)

In [10]:
si.map.save_to_file('Read_back_values.txt')

In [11]:
si.map.compare_values_sets(clock_builder_values, initial_values)

array([[  0.,  nan,   0.,   1.],
       [  1.,  nan,   0.,   1.],
       [  2.,  83.,   0.,   1.],
       [  3.,   0.,   0.,   0.],
       [  4.,  32.,  nan,   1.],
       [  7.,   1.,  nan,   1.],
       [  9.,  nan,   0.,   1.],
       [ 15.,   0.,   0.,   0.],
       [ 16.,  15.,  79.,   1.],
       [ 17.,  15.,  79.,   1.],
       [ 18.,  15.,  79.,   1.],
       [ 19., 140.,  79.,   1.],
       [ 20., 140.,  79.,   1.],
       [ 21., 140.,  79.,   1.],
       [ 22., 140.,  79.,   1.],
       [ 23., 140.,  79.,   1.],
       [ 24.,  nan,   0.,   1.],
       [ 25.,  nan,   0.,   1.],
       [ 26.,   0.,   0.,   0.],
       [ 27.,   1.,   1.,   0.],
       [ 28.,   0.,   0.,   0.],
       [ 29.,  16.,  16.,   0.],
       [ 30.,   0.,   0.,   0.],
       [ 31.,   0.,   0.,   0.],
       [ 32.,   0.,   0.,   0.],
       [ 33.,   0.,   0.,   0.],
       [ 34.,  nan,   0.,   1.],
       [ 35.,  nan,   1.,   1.],
       [ 36.,  nan,   0.,   1.],
       [ 37.,  nan,  16.,   1.],
       [ 3

In [12]:
si.map.compare_values_sets(initial_values, read_back_values) 

array([[  0.,   0.,  17.,   1.],
       [  1.,   0., 240.,   1.],
       [  2.,   0.,   0.,   0.],
       [  3.,   0.,   0.,   0.],
       [  9.,   0.,   0.,   0.],
       [ 15.,   0.,   0.,   0.],
       [ 16.,  79.,  79.,   0.],
       [ 17.,  79.,  79.,   0.],
       [ 18.,  79.,  79.,   0.],
       [ 19.,  79.,  79.,   0.],
       [ 20.,  79.,  79.,   0.],
       [ 21.,  79.,  79.,   0.],
       [ 22.,  79.,  79.,   0.],
       [ 23.,  79.,  79.,   0.],
       [ 24.,   0.,   0.,   0.],
       [ 25.,   0.,   0.,   0.],
       [ 26.,   0.,   0.,   0.],
       [ 27.,   1.,   1.,   0.],
       [ 28.,   0.,   0.,   0.],
       [ 29.,  16.,  16.,   0.],
       [ 30.,   0.,   0.,   0.],
       [ 31.,   0.,   0.,   0.],
       [ 32.,   0.,   0.,   0.],
       [ 33.,   0.,   0.,   0.],
       [ 34.,   0.,   0.,   0.],
       [ 35.,   1.,   1.,   0.],
       [ 36.,   0.,   0.,   0.],
       [ 37.,  16.,  16.,   0.],
       [ 38.,   0.,   0.,   0.],
       [ 39.,   0.,   0.,   0.],
       [ 4

In [13]:
matchs = si.find_integer_dividers(freq_desired = 45e6, even_only = True, torance_hz = 1) 
matchs

[((18, 10, 1), (25000000, 450000000, 45000000, 45000000)),
 ((36, 10, 2), (25000000, 900000000, 90000000, 45000000)),
 ((36, 20, 1), (25000000, 900000000, 45000000, 45000000)),
 ((54, 30, 1), (25000000, 1350000000, 45000000, 45000000)),
 ((72, 10, 4), (25000000, 1800000000, 180000000, 45000000)),
 ((72, 20, 2), (25000000, 1800000000, 90000000, 45000000)),
 ((72, 40, 1), (25000000, 1800000000, 45000000, 45000000)),
 ((90, 50, 1), (25000000, 2250000000, 45000000, 45000000))]

In [35]:
desired_clock_freqs = (45e6, 30e5, 90e6)

common_pll_dividers, freqs_pll_dividers, freqs_matches = si.find_integer_pll_dividers_for_clocks(desired_clock_freqs)
common_pll_dividers, freqs_pll_dividers

({36, 72},
 [{18, 36, 54, 72, 90},
  {18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90},
  {36, 72}])

In [15]:
clk = si.clocks[0]

In [16]:
clk.set_frequency(25e6)

True

In [17]:
clk.freq

25000000

In [18]:
si.plls[0].set_frequency(900e6)

True

In [19]:
si.plls[0].divider

36.0

In [20]:
# _i2c.scan()

In [21]:
si.enable(True)

In [22]:
si.enable(False)

In [23]:
si.clocks[0].enable()

In [24]:
si.read_all_registers()

[(0, 'Device_Status', 17),
 (1, 'Interrupt_Status_Sticky', 240),
 (2, 'Interrupt_Status_Mask', 0),
 (3, 'Output_Enable_Control', 254),
 (9, 'OEB_Pin_Enable_Control_Mask', 0),
 (15, 'PLL_Input_Source', 0),
 (16, 'CLK0_Control', 79),
 (17, 'CLK1_Control', 207),
 (18, 'CLK2_Control', 207),
 (19, 'CLK3_Control', 207),
 (20, 'CLK4_Control', 207),
 (21, 'CLK5_Control', 207),
 (22, 'CLK6_Control', 207),
 (23, 'CLK7_Control', 207),
 (24, 'CLK3_0_Disable_State', 0),
 (25, 'CLK7_4_Disable_State', 0),
 (26, 'Multisynth_NA_Parameters_26', 0),
 (27, 'Multisynth_NA_Parameters_27', 1),
 (28, 'Multisynth_NA_Parameters_28', 0),
 (29, 'Multisynth_NA_Parameters_29', 16),
 (30, 'Multisynth_NA_Parameters_30', 0),
 (31, 'Multisynth_NA_Parameters_31', 0),
 (32, 'Multisynth_NA_Parameters_32', 0),
 (33, 'Multisynth_NA_Parameters_33', 0),
 (34, 'Multisynth_NB_Parameters_34', 0),
 (35, 'Multisynth_NB_Parameters_35', 1),
 (36, 'Multisynth_NB_Parameters_36', 0),
 (37, 'Multisynth_NB_Parameters_37', 16),
 (38, 'Mul

In [31]:
si.status.print()


<< Device_Status >>:  ('0x11', '0b10001')
[ SYS_INIT ]   :  0
[ LOL_B ]      :  0
[ LOL_A ]      :  0
[ LOS_CLKIN ]  :  1
[ LOS_XTAL ]   :  0
[ Reserved_2 ] :  0
[ REVID ]      :  1


10

In [32]:
si.ready

True

In [27]:
si.interrupts.stickys.print()


<< Interrupt_Status_Sticky >>:  ('0xf0', '0b11110000')
[ SYS_INIT_STKY ]  :  1
[ LOL_B_STKY ]     :  1
[ LOL_A_STKY ]     :  1
[ LOS_CLKIN_STKY ] :  1
[ LOS_XTAL_STKY ]  :  0
[ Reserved_0 ]     :  0


14

## Member functions test

In [28]:
for f in dir(cls):
    if not f.startswith('_'):
        print('si.{}()'.format(f))

si.CLKIN_DIVIDERS()
si.CLOCK_SOURCEs()
si.CRYSTAL_INTERNAL_LOAD_CAPACITANCEs()
si.DEBUG_MODE()
si.DEBUG_MODE_PRINT_REGISTER()
si.DEBUG_MODE_SHOW_BUS_DATA()
si.DENOMINATOR_BITS()
si.DISABLE_STATEs()
si.FREQ_MCLK()
si.I2C_ADDRESS()
si.INTEGER_ONLY_multisynths()
si.N_HIGH_FREQUENCY_multisynths()
si.N_OUTPUT_CLOCKS()
si.N_PLLS()
si.N_USED_OUTPUT_CLOCKS()
si.N_multisynths_WITH_OUTPUT_SKEW()
si.OUTPUT_STRENGTHs()
si.PLLs()
si.POW_2_DENOMINATOR_BITS()
si.READ_ONLY_REGISTERS()
si.R_DIVIDERs()
si.close()
si.current_frequency()
si.current_phase()
si.do()
si.do_on_devices()
si.enable()
si.enable_output()
si.enable_output_channel()
si.find_integer_dividers()
si.find_integer_pll_dividers_for_clocks()
si.init()
si.is_virtual_device()
si.load_registers()
si.pause()
si.print()
si.read_all_registers()
si.ready()
si.reset()
si.restore_clocks_freqs()
si.resume()
si.revision()
si.set_frequency()
si.set_phase()
si.start()
si.status()
si.stop()
si.update()
si.write_all_registers()


In [29]:
si.apply_signal()
si.current_frequency()
si.current_phase()
si.do()
si.do_on_devices()
si.enable()
si.enable_output()
si.enable_output_channel()
si.enable_output_integer_divider()
si.enable_phase_offset()
si.enable_pll_integer_divider()
si.enable_spread_spectrum()
si.enabled()
si.freq_resolution()
si.frequency()
si.get_freq_vco_clkin()
si.get_freq_vco_xtal()
si.get_multisynth_nb_parameter()
si.get_output_multisynth_divider()
si.get_pll_multisynth_divider()
si.init()
si.is_abc_even_integer()
si.is_even_integer()

si.phase_resolution()
si.power_down_all_outputs()
si.power_down_clock_output()
si.power_down_output()
si.print()
si.reset()
si.reset_pll()
si.reset_plls()

si.select_freq_source()
si.select_phase_source()
si.set_center_spread()
si.set_clock_disable_state()
si.set_clock_invert()
si.set_cmos_clock_source()
si.set_divide_by_4()
si.set_down_spread()
si.set_frequency()
si.set_interrupt_mask()
si.set_interrupts_mask()
si.set_output_driver_source()
si.set_output_multisynth_source()
si.set_phase()
si.set_pll_input_source()
si.set_r_divider()
si.set_ss_parameters()
si.set_vcoa()
si.set_vcob()
si.set_vcxo_paramenters()
si.set_xtal_source()
si.shape()
si.start()
si.pause()
si.resume()
si.stop()
si.update()

si.close()

AttributeError: 'Si5351' object has no attribute 'apply_signal'

In [ ]:
si.phase_resolution()